In [55]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [29]:
data = pd.read_table("smsspamcollection/SMSSpamCollection", header=None, names=['label', 'text'])

In [30]:
data.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [31]:
data['label'] = data['label'].apply(lambda x: int(x == 'spam'))

In [32]:
data.head(10)

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [33]:
X_texts = data['text'].values
y = data['label'].values

In [34]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_texts)

In [38]:
scores = cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')

In [39]:
print(np.average(scores))

0.932640298361


In [43]:
X_test_texts = np.array(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                         "FreeMsg: Txt: claim your reward of 3 hours talk time",
                         "Have you visited the last lecture on physics?",
                         "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                         "Only 99$"])
X_test = vectorizer.transform(X_test_texts)

In [44]:
model = LogisticRegression()
model = model.fit(X, y)
y_pred = model.predict(X_test)

In [45]:
print(y_pred)

[1 1 0 0 0]


In [47]:
cross_val_scores = []

In [48]:
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    X = vectorizer.fit_transform(X_texts)
    cross_val_scores.append(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

In [49]:
print(cross_val_scores)

[0.82242206641871329, 0.72501615554673771, 0.92513825586488374]


In [51]:
cross_val_scores_nb = []

In [52]:
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    X = vectorizer.fit_transform(X_texts)
    cross_val_scores_nb.append(np.average(cross_val_score(MultinomialNB(), X, y, cv=10, scoring='f1')))

In [53]:
print(cross_val_scores_nb)

[0.64550151779854426, 0.37871948524573595, 0.88848596560610016]


In [56]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(X_texts)
print(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

0.852859955417


In [57]:
vectorizer = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(X_texts)
print(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

0.932640298361
